In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

data\submission.csv
data\test.csv
data\train.csv


In [12]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
def RMSLE(pred,actual):
    return np.sqrt(np.mean(np.power((np.log(pred+1)-np.log(actual+1)),2)))

In [16]:
pd.set_option('mode.chained_assignment', None)
test = pd.read_csv("data/test.csv")
train = pd.read_csv("data/train.csv")
train['Province_State'].fillna('', inplace=True)
test['Province_State'].fillna('', inplace=True)
train['Date'] =  pd.to_datetime(train['Date'])
test['Date'] =  pd.to_datetime(test['Date'])
train = train.sort_values(['Country_Region','Province_State','Date'])
test = test.sort_values(['Country_Region','Province_State','Date'])

In [17]:
train[train['Country_Region'] == 'Ukraine'].tail()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
23435,33364,,Ukraine,2020-04-06,1319.0,38.0
23436,33365,,Ukraine,2020-04-07,1462.0,45.0
23437,33366,,Ukraine,2020-04-08,1668.0,52.0
23438,33367,,Ukraine,2020-04-09,1892.0,57.0
23439,33368,,Ukraine,2020-04-10,2203.0,69.0


In [18]:

feature_day = [1,20,50,100,200,500,1000,2000,5000,10000]
def CreateInput(data):
    feature = []
    for day in feature_day:
        #Get information in train data
        data.loc[:,'Number day from ' + str(day) + ' case'] = 0
        if (train[(train['Country_Region'] == country) & (train['Province_State'] == province) & (train['ConfirmedCases'] < day)]['Date'].count() > 0):
            fromday = train[(train['Country_Region'] == country) & (train['Province_State'] == province) & (train['ConfirmedCases'] < day)]['Date'].max()        
        else:
            fromday = train[(train['Country_Region'] == country) & (train['Province_State'] == province)]['Date'].min()       
        for i in range(0, len(data)):
            if (data['Date'].iloc[i] > fromday):
                day_denta = data['Date'].iloc[i] - fromday
                data['Number day from ' + str(day) + ' case'].iloc[i] = day_denta.days 
        feature = feature + ['Number day from ' + str(day) + ' case']
    
    return data[feature]
pred_data_all = pd.DataFrame()
for country in train['Country_Region'].unique():
    for province in train[(train['Country_Region'] == country)]['Province_State'].unique():
        df_train = train[(train['Country_Region'] == country) & (train['Province_State'] == province)]
        df_test = test[(test['Country_Region'] == country) & (test['Province_State'] == province)]
        X_train = CreateInput(df_train)
        y_train_confirmed = df_train['ConfirmedCases'].ravel()
        y_train_fatalities = df_train['Fatalities'].ravel()
        X_pred = CreateInput(df_test)
        
        # Only train above 50 cases
        for day in sorted(feature_day,reverse = True):
            feature_use = 'Number day from ' + str(day) + ' case'
            idx = X_train[X_train[feature_use] == 0].shape[0]     
            if (X_train[X_train[feature_use] > 0].shape[0] >= 50):
                break
      #  print(country, province, idx, day)
        adjusted_X_train = X_train[idx:][feature_use].values.reshape(-1, 1)
        adjusted_y_train_confirmed = y_train_confirmed[idx:]
        adjusted_y_train_fatalities = y_train_fatalities[idx:]#.values.reshape(-1, 1)
        idx = X_pred[X_pred[feature_use] == 0].shape[0]    
        adjusted_X_pred = X_pred[idx:][feature_use].values.reshape(-1, 1)
        
        pred_data = test[(test['Country_Region'] == country) & (test['Province_State'] == province)]
        max_train_date = train[(train['Country_Region'] == country) & (train['Province_State'] == province)]['Date'].max()
        min_test_date = pred_data['Date'].min()
        #The number of day forcast
        #pred_data[pred_data['Date'] > max_train_date].shape[0]
        #model = SimpleExpSmoothing(adjusted_y_train_confirmed).fit()
        #model = Holt(adjusted_y_train_confirmed).fit()
        #model = Holt(adjusted_y_train_confirmed, exponential=True).fit()
        #model = Holt(adjusted_y_train_confirmed, exponential=True, damped=True).fit()
        model = ExponentialSmoothing(adjusted_y_train_confirmed, trend = 'additive').fit()
        y_hat_confirmed = model.forecast(pred_data[pred_data['Date'] > max_train_date].shape[0])
        y_train_confirmed = train[(train['Country_Region'] == country) & (train['Province_State'] == province) & (train['Date'] >=  min_test_date)]['ConfirmedCases'].values
        y_hat_confirmed = np.concatenate((y_train_confirmed,y_hat_confirmed), axis = 0)
               
        #model = Holt(adjusted_y_train_fatalities).fit()
        model = ExponentialSmoothing(adjusted_y_train_fatalities, trend = 'additive').fit()
        y_hat_fatalities = model.forecast(pred_data[pred_data['Date'] > max_train_date].shape[0])
        y_train_fatalities = train[(train['Country_Region'] == country) & (train['Province_State'] == province) & (train['Date'] >=  min_test_date)]['Fatalities'].values
        y_hat_fatalities = np.concatenate((y_train_fatalities,y_hat_fatalities), axis = 0)
      #  print(pred_data.shape,y_hat_confirmed.shape)
        
        pred_data['ConfirmedCases_hat'] =  y_hat_confirmed
        pred_data['Fatalities_hat'] = y_hat_fatalities
        pred_data_all = pred_data_all.append(pred_data)

df_val = pd.merge(pred_data_all,train[['Date','Country_Region','Province_State','ConfirmedCases','Fatalities']],on=['Date','Country_Region','Province_State'], how='left')
df_val

,ForecastId,Province_State,Country_Region,Date,ConfirmedCases_hat,Fatalities_hat,ConfirmedCases,Fatalities
0,1,,Afghanistan,2020-04-02,273.000000,6.000000,273.0,6.0
1,2,,Afghanistan,2020-04-03,281.000000,6.000000,281.0,6.0
2,3,,Afghanistan,2020-04-04,299.000000,7.000000,299.0,7.0
3,4,,Afghanistan,2020-04-05,349.000000,7.000000,349.0,7.0
4,5,,Afghanistan,2020-04-06,367.000000,11.000000,367.0,11.0
...,...,...,...,...,...,...,...,...
13454,13455,,Zimbabwe,2020-05-10,30.142852,7.090909,NaN,NaN
13455,13456,,Zimbabwe,2020-05-11,30.714280,7.227273,NaN,NaN
13456,13457,,Zimbabwe,2020-05-12,31.285709,7.363636,NaN,NaN
13457,13458,,Zimbabwe,2020-05-13,31.857137,7.500000,NaN,NaN


In [20]:
country = "Japan" #"Ukraine" # "Sweden"
df_country = df_val[df_val['Country_Region'] == country].groupby(['Date','Country_Region']).sum().reset_index()
df_country

,Date,Country_Region,ForecastId,ConfirmedCases_hat,Fatalities_hat,ConfirmedCases,Fatalities
0,2020-04-02,Japan,6365,2495.000000,62.000000,2495.0,62.0
1,2020-04-03,Japan,6366,2617.000000,63.000000,2617.0,63.0
2,2020-04-04,Japan,6367,3139.000000,77.000000,3139.0,77.0
3,2020-04-05,Japan,6368,3139.000000,77.000000,3139.0,77.0
4,2020-04-06,Japan,6369,3654.000000,85.000000,3654.0,85.0
5,2020-04-07,Japan,6370,3906.000000,92.000000,3906.0,92.0
6,2020-04-08,Japan,6371,4257.000000,93.000000,4257.0,93.0
7,2020-04-09,Japan,6372,4667.000000,94.000000,4667.0,94.0
8,2020-04-10,Japan,6373,5530.000000,99.000000,5530.0,99.0
9,2020-04-11,Japan,6374,5914.127273,102.707475,0.0,0.0


In [21]:
country = "Ukraine"
df_country = df_val[df_val['Country_Region'] == country].groupby(['Date','Country_Region']).sum().reset_index()
idx = df_country[((df_country['ConfirmedCases'].isnull() == False) & (df_country['ConfirmedCases'] > 0))].shape[0]
fig = px.line(df_country, x="Date", y="ConfirmedCases_hat", title='Total Cases of ' + df_country['Country_Region'].values[0])
fig.add_scatter(x=df_country['Date'][0:idx], y=df_country['ConfirmedCases'][0:idx], mode='lines', name="Actual", showlegend=False)
fig.show()

fig = px.line(df_country, x="Date", y="Fatalities_hat", title='Total Fatalities of ' + df_country['Country_Region'].values[0])
fig.add_scatter(x=df_country['Date'][0:idx], y=df_country['Fatalities'][0:idx], mode='lines', name="Actual", showlegend=False)
fig.show()

In [22]:
df_total = df_val.groupby(['Date']).sum().reset_index()

idx = df_total[((df_total['ConfirmedCases'].isnull() == False) & (df_total['ConfirmedCases'] > 0))].shape[0]
fig = px.line(df_total, x="Date", y="ConfirmedCases_hat", title='Total Cases of World')
fig.add_scatter(x=df_total['Date'][0:idx], y=df_total['ConfirmedCases'][0:idx], mode='lines', name="Actual", showlegend=False)
fig.show()

fig = px.line(df_total, x="Date", y="Fatalities_hat", title='Total Fatalities of World')
fig.add_scatter(x=df_total['Date'][0:idx], y=df_total['Fatalities'][0:idx], mode='lines', name="Actual", showlegend=False)
fig.show()

In [9]:
submission = df_val[['ForecastId','ConfirmedCases_hat','Fatalities_hat']]
submission.columns = ['ForecastId','ConfirmedCases','Fatalities']
submission.to_csv('submission.csv', index=False)

In [10]:
submission

,ForecastId,ConfirmedCases,Fatalities
0,1,94.000000,4.000000
1,2,110.000000,4.000000
2,3,110.000000,4.000000
3,4,120.000000,4.000000
4,5,170.000000,4.000000
...,...,...,...
13153,13154,25.444442,4.382973
13154,13155,25.999997,4.480759
13155,13156,26.555553,4.578545
13156,13157,27.111108,4.676331
